<a href="https://colab.research.google.com/github/ogustaferreira/CoordinateAI/blob/main/CoordinateBot_Seu_ChatBot_de_Astronomia_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install -U -q google-generativeai

In [ ]:
!pip install wikipedia

In [77]:
import wikipedia
wikipedia.set_lang("pt")

import pandas as pd
import numpy as np
import google.generativeai as genai

from google.colab import userdata
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

In [143]:
def get_embedding_model():
  """Encontra um modelo que suporta o método 'embedContent'."""
  for m in genai.list_models():
    if 'embedContent' in m.supported_generation_methods:
      return m.name
  raise ValueError("Nenhum modelo encontrado com suporte a 'embedContent'.")

model_name = get_embedding_model()
print(f"Usando o modelo: {model_name}")

Usando o modelo: models/embedding-001


In [ ]:
#Exemplo de embedding
title = "CoordinateBot ou CoordinateAI, um bot focado em Astronomia!"
sample_text = ("Título: CoordinateBot ou CoordinateAI, um bot focado em Astronomia!"
    "\n"
    "Artigo completo:\n"
    "\n"
    "Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

In [156]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Título": "Observação do Céu Noturno",
    "Conteúdo": "A observação do céu noturno é um passatempo fascinante que permite contemplar a vastidão do universo. Para uma experiência ideal, siga estas dicas:\n\n* **Encontre um local escuro:** Fuja das luzes da cidade para evitar a poluição luminosa.\n* **Adapte seus olhos à escuridão:** Aguarde cerca de 30 minutos para que seus olhos se ajustem à baixa luminosidade, permitindo a visualização de objetos mais tênues.\n* **Use um mapa celeste ou aplicativo:** Identifique estrelas, constelações e planetas com auxílio de um mapa celeste ou aplicativo de astronomia.\n* **Comece com binóculos:** Binóculos são uma ótima opção para iniciantes, oferecendo um campo de visão mais amplo que telescópios.\n* **Explore diferentes objetos celestes:** Observe planetas brilhantes como Vênus e Marte, aglomerados estelares como as Plêiades, e nebulosas como a Nebulosa de Órion.\n* **Seja paciente e aproveite a experiência:** A astronomia exige tempo e paciência. Relaxe, contemple a beleza do cosmos e desfrute da jornada."
}

DOCUMENT2 = {
    "Título": "Telescópios: Abrindo as Janelas do Universo",
    "Conteúdo": "Telescópios são instrumentos poderosos que nos permitem observar objetos celestes distantes com detalhes incríveis. Existem diversos tipos de telescópios, cada um com suas vantagens:\n\n* **Refratores:** Utilizam lentes para focar a luz, geralmente sendo mais compactos e fáceis de usar.\n* **Refletores:** Empregam espelhos para coletar e focar a luz, sendo ideais para observar objetos de brilho fraco.\n* **Catadióptricos:** Combinam lentes e espelhos, oferecendo um bom equilíbrio entre desempenho e portabilidade.\n\nAo escolher um telescópio, considere fatores como abertura (diâmetro da lente ou espelho), distância focal e montagem. Consulte recursos online e converse com astrônomos experientes para tomar a melhor decisão."
}

DOCUMENT3 = {
    "Título": "Explorando o Sistema Solar",
    "Conteúdo": "O Sistema Solar é um lugar fascinante, repleto de planetas, luas, asteroides e cometas. Mergulhe em uma jornada de descobertas:\n\n* **Planetas Internos:** Mercúrio, Vênus, Terra e Marte, compostos principalmente de rocha e metal.\n* **Planetas Externos:** Gigantes gasosos como Júpiter, Saturno, Urano e Netuno, com atmosferas densas e sistemas de anéis.\n* **Cinturão de Asteroides:** Localizado entre Marte e Júpiter, abriga milhares de asteroides de diferentes tamanhos e composições.\n* **Cinturão de Kuiper:** Região além de Netuno, lar de objetos transnetunianos, incluindo Plutão.\n\nExplore as características únicas de cada planeta, suas luas, e os mistérios do Sistema Solar. A aventura te espera!"
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [146]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Planetas do Sistema Solar,O Sistema Solar é composto por oito planetas: ...
1,Estrelas e Galáxias,As estrelas são corpos celestes gigantes que e...
2,O Big Bang,A teoria do Big Bang é o modelo cosmológico pr...


In [147]:
model = "models/embedding-001"

In [148]:
def gerar_embedding(title, text, model):
  """Gera um embedding para o título e o texto fornecidos."""
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

df["Embeddings"] = df.apply(lambda row: gerar_embedding(row["Titulo"], row["Conteudo"], model_name), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Planetas do Sistema Solar,O Sistema Solar é composto por oito planetas: ...,"[-0.0024084046, -0.042832825, -0.06734525, 0.0..."
1,Estrelas e Galáxias,As estrelas são corpos celestes gigantes que e...,"[0.049588293, -0.040754307, -0.031406455, -0.0..."
2,O Big Bang,A teoria do Big Bang é o modelo cosmológico pr...,"[0.024029301, -0.06565786, -0.06176271, 0.0390..."


In [158]:
def gerar_embedding_consulta(consulta, model):
  """Gera um embedding para a consulta fornecida."""
  return genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

def buscar_documento_semelhante(embedding_consulta, embeddings_documentos, limiar=0.8):
  """Encontra o documento com o embedding mais semelhante ao da consulta."""
  produtos_escalares = np.dot(np.stack(embeddings_documentos), embedding_consulta)
  indice = np.argmax(produtos_escalares)

  # Verifica se a similaridade é maior que o limiar
  if produtos_escalares[indice] > limiar:
    return indice
  else:
    return None  # Retorna None se a similaridade for baixa

#Faça sua consulta aos documentos!
consulta = "Quais são os 8 planetas do sistema solar"
embedding_consulta = gerar_embedding_consulta(consulta, model_name)
indice_documento = buscar_documento_semelhante(embedding_consulta, df["Embeddings"])

def buscar_na_wikipedia(consulta):
  """Realiza uma busca na Wikipedia e retorna o resumo do primeiro resultado."""
  try:
    resultados = wikipedia.search(consulta)
    if resultados:
      pagina = wikipedia.page(resultados[0])
      return pagina.summary
    else:
      return "Nenhum resultado encontrado na Wikipedia."
  except wikipedia.exceptions.DisambiguationError as e:
    return f"Múltiplas opções encontradas. Seja mais específico: {e.options}"
  except wikipedia.exceptions.PageError as e:
    return f"Página não encontrada na Wikipedia: {e}"

# Função para gerar resposta com Gemini
def gerar_resposta(prompt, model="gemini-1.0-pro", temperature=0.2, max_output_tokens=1024):
  """Gera uma resposta textual usando o modelo Gemini."""
  generation_config = {
      "temperature": temperature,
      "max_output_tokens": max_output_tokens
  }
  model = genai.GenerativeModel(model, generation_config=generation_config)
  response = model.generate_content(prompt)
  return response.text

# Loop principal do bot
while True:
  consulta = input("Faça uma pergunta sobre astronomia: ")
  if consulta.lower() == "sair":
    break

  # Tenta encontrar a resposta nos documentos
  embedding_consulta = gerar_embedding_consulta(consulta, model="models/embedding-001")
  indice_documento = buscar_documento_semelhante(embedding_consulta, df["Embeddings"])

  # Se encontrar um documento relevante
  if indice_documento is not None:
      documento_relevante = df.iloc[indice_documento]
      trecho = documento_relevante["Conteudo"]
      print(trecho)
      prompt = f"""Você é um bot de astronomia especialista e deve responder à pergunta usando a informação fornecida.

      Informação:
      {documento_relevante["Conteudo"]}

      Pergunta: {consulta}

      Resposta:"""

      resposta = gerar_resposta(prompt)

  # Se NÃO encontrar um documento relevante, busca na Wikipedia
  else:
      resposta_wikipedia = buscar_na_wikipedia(consulta)
      prompt = f"""Você é um bot de astronomia especialista.
      Usando a informação da Wikipedia abaixo, responda a pergunta de forma concisa e precisa.

      Informação da Wikipedia:
      {resposta_wikipedia}

      Pergunta: {consulta}

      Resposta:"""
      resposta = gerar_resposta(prompt, temperature=0.2)

  print(resposta)

Faça uma pergunta sobre astronomia: Sistema solar
O Sistema Solar é um conjunto de corpos celestes sob a influência gravitacional do Sol, incluindo planetas, planetas anões, asteroides, cometas e satélites naturais.
Faça uma pergunta sobre astronomia: Qual a distancia entre a terra e galaxia de andromeda?
2,54 milhões de anos-luz
Faça uma pergunta sobre astronomia: Constelações visiveis da terra
Crux (Cruzeiro do Sul)
Faça uma pergunta sobre astronomia: sair
